In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras import optimizers
from tensorflow.keras import optimizers
import tensorflow as tf
import pandas as pd
import numpy as np

**전처리하기**

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/dirty_mnist_2nd_answer.csv")
for i in range(50000):
  df["index"][i] = str(df["index"][i]).zfill(5) + ".png"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,00000.png,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,00001.png,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,00002.png,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,00003.png,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,00004.png,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
for i in range(5000):
  test_df["index"][i] = str(test_df["index"][i]) + ".png"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
test_df.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,50001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,50002.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,50003.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50004.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import cv2
from keras.preprocessing.image import img_to_array

columns=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]

def denoise(image):
  return cv2.fastNlMeansDenoisingColored(image, None, 23, 23, 7, 21)

datagen=ImageDataGenerator( rescale=1./255. ) 

test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
    dataframe=df[:40000],
    directory="/content/drive/MyDrive/dirty_mnist",
    x_col="index",
    y_col=columns,
    color_mode = 'rgb',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw", 
    preprocessing_function = denoise,
    target_size=(150,150))

valid_generator=test_datagen.flow_from_dataframe(
    dataframe=df[40000:50000],
    directory="/content/drive/MyDrive/dirty_mnist",
    x_col="index",
    y_col=columns,
    color_mode = 'rgb',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    preprocessing_function = denoise,
    target_size=(150,150))


test_generator=test_datagen.flow_from_dataframe(
    test_df,
    directory="/content/drive/MyDrive/test_dirty_mnist",
    x_col="index",
    color_mode = "grayscale",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    preprocessing_function = denoise,
    target_size=(150 , 150))


Found 40000 validated image filenames.
Found 10000 validated image filenames.
Found 5000 validated image filenames.


In [ ]:
# score 함수 (정기님꺼 참고)
import keras.backend as K
def score(y_true, y_pred):
    y_pred = K.cast(K.greater(y_pred, 0.5), 'float32')
    equal = K.sum(K.cast(K.equal(y_true, y_pred), 'int32'))
    return equal/tf.size(y_true)

**RetNet50**

In [ ]:
from keras.models import Model
from keras.applications import ResNet50

height = 150 #dimensions of image
width = 150
channel = 3 #RGB

# Create pre-trained ResNet50 without top layer
model_resnet = ResNet50(include_top=False, weights="imagenet", input_shape=(height, width, channel))

conv1 = Conv2D(32, (3,3), activation = 'relu')(model_resnet.output)
pool1 = MaxPooling2D(2,2)(conv1)
bn1 = BatchNormalization(axis=-1)(pool1)
drop1 = Dropout(0.2)(bn1)

flatten1 = Flatten()(drop1)
fc2 = Dense(26, activation='sigmoid')(flatten1)

additional_model2 = Model(inputs=model_resnet.input, outputs=fc2)


additional_model2.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=[score])

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
additional_model2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
1250/1250 [==============================] - 8387s 7s/step - loss: 0.7495 - score: 0.5052 - val_loss: 0.7032 - val_score: 0.5177
Epoch 2/3
1250/1250 [==============================] - 1650s 1s/step - loss: 0.7105 - score: 0.5267 - val_loss: 1.2036 - val_score: 0.5211
Epoch 3/3
1250/1250 [==============================] - 1651s 1s/step - loss: 0.7051 - score: 0.5376 - val_loss: 0.6967 - val_score: 0.5272


In [ ]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size #런타임 끊겼는데, 이전에 최종적으로 실행한 것과 validation accuracy가 비슷했습니다.
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

NameError: ignored

In [ ]:
pred_bool = (pred >0.5)

In [ ]:
predictions = pred_bool.astype(int)
columns=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=columns)
results["index"]=test_generator.filenames
ordered_cols=["index"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv("results.csv",index=False)

In [ ]:
for i in range(5000):
  results['index'][i] = str(results['index'][i][0:5])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
results.to_csv("results.csv",index=False)